In [1]:
import torch
from transformers import pipeline, BertConfig, BertModel

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.zeros(1).cuda()

AssertionError: Torch not compiled with CUDA enabled